In [1]:
from rl import ValueAction

value_action = ValueAction(100, 128)

In [12]:
import numpy as np
maze_map = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
            [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
            [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
            [1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
            [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
            [1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
            [1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
            [1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
maze_map = np.array(maze_map)

In [17]:
np.where(maze_map[:, x] == 1)

(array([0, 9], dtype=int64),)

In [1]:
from train import train_dql, ValueAction, Environment, EPolicy
from environment import WALL_M, Point, Agent
from agents import AgentVision
import numpy as np

# Maze Parameters
STATE_SIZE = 500 # observation size * types of tiles
#hyper parameters
BATCH_SIZE = 128
BUFFER_SIZE = 10000
DIM = 0.9
ALPHA = 0.001
NUM_EPISODES = 8000
MAX_STEPS = 100
TAU = 0.1
# First 250
# EPSILON = 0.6
# EPSILON_DECAY = 0.9999
# After 250
EPSILON = 0.1
EPSILON_DECAY = 0.99999
DROPOUT = 0.1
STEPS_PER_UPDATE = 100
ARCHITECTURE = 'one_hidden'
N = 1
value_action = ValueAction(state_size=STATE_SIZE, architecture=ARCHITECTURE, batch_size=BATCH_SIZE, buffer_size=BUFFER_SIZE, steps_per_update=STEPS_PER_UPDATE, dim=DIM, alpha=ALPHA, tau=TAU, n=N)


Using cuda device
pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
value_action.load('../one_check.pth')

c:\Users\mokrota\Documents\GitHub\RL_projects\dql\rl.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(path)


In [4]:
for name, param in value_action.qnetwork_other.named_parameters():
    print(f"Layer: {name} | Weights: {param.data}")


Layer: network.0.weight | Weights: tensor([[ 0.0290,  0.0444,  0.0091,  ...,  0.0018, -0.0292, -0.0370],
        [ 0.0216, -1.1514, -0.0066,  ...,  0.0247, -0.0801,  0.0206],
        [-0.0255, -0.0308,  0.0298,  ..., -0.0382, -0.0626, -0.0132],
        ...,
        [-0.0161, -0.6540,  0.0114,  ..., -0.0103, -0.1775, -0.0074],
        [ 0.0259,  0.0311,  0.0046,  ..., -0.0115, -0.0127, -0.0112],
        [-0.0123,  0.4271, -0.0012,  ...,  0.0305,  0.4410, -0.0146]],
       device='cuda:0')
Layer: network.0.bias | Weights: tensor([-7.8636e-02, -1.5917e-01, -1.5613e-02, -2.2471e-01, -9.7867e-02,
        -6.3118e-02,  1.9823e-03, -3.5478e-02, -6.3982e-02,  2.7876e-02,
        -5.7496e-02, -8.4765e-02, -9.0141e-02, -9.5881e-02, -1.0661e-01,
        -3.3833e-01, -2.6717e-01,  6.3523e-03,  6.2787e-03,  2.8073e-02,
        -4.0217e-02, -3.4348e-01, -4.6361e-02, -5.9466e-02, -1.9589e-01,
        -2.4263e-02, -2.3970e-01, -1.4956e-02, -1.5098e-01, -5.1749e-01,
        -3.2552e-02,  7.2185e-03, -6

In [20]:
y, x = 1, 3
wall_u = np.where(maze_map[:, x] == 1)[0]
wall_u = y - wall_u[0] - 1
wall_u

0

: 

In [7]:
%pip install keyboard

Note: you may need to restart the kernel to use updated packages.


In [2]:
def one_hot_encode_tile(tile):
    if tile == 1:
        return [1, 0, 0, 0]  # wall
    elif tile == 0:
        return [0, 1, 0, 0]  # empty
    elif tile == 2:
        return [0, 0, 1, 0]  # subgoal
    elif tile == 3:
        return [0, 0, 0, 1]  # goal

one_hot_maze = np.array([[one_hot_encode_tile(tile) for tile in row] for row in maze_map])


In [3]:
one_hot_maze

array([[[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
  

In [6]:
one_hot_maze.flatten()

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [4]:
import torch
maze_tensor = torch.tensor(one_hot_maze).float()


In [5]:
maze_tensor

tensor([[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 